<a href="https://colab.research.google.com/github/felixchiuman/Stock-Market-Analysis/blob/main/stock_market_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#library

!pip install python-telegram-bot yfinance pandas_ta nest_asyncio mplfinance
from google.colab import userdata

Bot Code

In [ ]:
import logging
import yfinance as yf
import pandas as pd
import pandas_ta as ta
import nest_asyncio
import asyncio
import os
import time
import datetime
import numpy as np
from telegram import Update
from telegram.constants import ParseMode
from telegram.ext import ApplicationBuilder, ContextTypes, CommandHandler

# --- 1. CONFIGURATION ---
nest_asyncio.apply()
BOT_TOKEN = userdata.get('botToken')

MEME_URLS = {
    "BUY": "https://i.imgflip.com/1ur9b0.jpg",
    "SELL": "https://i.imgflip.com/261o3j.jpg",
    "WAIT": "https://i.imgflip.com/26am.jpg"
}

# --- 2. THE "CHUNKING" DATA FETCHER (5-Year Deep Scan) ---
def get_chunked_data(ticker, years_back=5):
    """
    Downloads data year-by-year to prevent timeouts.
    REMOVED: Custom Session (causes conflict with new yfinance update).
    """
    print(f"🔄 Deep Scan: {ticker} ({years_back} years)...")
    frames = []
    current_year = datetime.date.today().year

    for i in range(years_back):
        year = current_year - i
        start_date = f"{year}-01-01"
        end_date = f"{year}-12-31"
        if year == current_year:
            end_date = datetime.date.today().strftime("%Y-%m-%d")

        try:
            # FIX: Do not pass 'session=...'. Let yfinance handle it.
            stock = yf.Ticker(ticker)
            df_chunk = stock.history(start=start_date, end=end_date)

            if not df_chunk.empty:
                frames.append(df_chunk)
                # print(f"   ✅ Got {year}")

            # Vital: Sleep to avoid hitting Yahoo rate limits since we have no cache
            time.sleep(0.5)

        except Exception as e:
            print(f"   ⚠️ Skipped {year}: {e}")

    if not frames: return pd.DataFrame()

    # Combine and Clean
    full_df = pd.concat(frames).sort_index()
    full_df = full_df[~full_df.index.duplicated(keep='first')]
    return full_df

# --- 3. BACKTEST ENGINE ---
def calculate_backtest_metrics(df):
    wins = 0; losses = 0; gross_profit = 0.0; gross_loss = 0.0; total_trades = 0

    closes = df['Close'].values; means = df['Mean'].values
    std_devs = df['StdDev'].values; z_scores = df['Z_Score'].values
    highs = df['High'].values; lows = df['Low'].values

    # Loop through history (skip last 15 days)
    for i in range(len(df) - 15):
        if z_scores[i] < -2.0: # Buy Signal
            total_trades += 1
            entry_price = closes[i]
            target_price = means[i]
            stop_price = entry_price - std_devs[i]

            for j in range(1, 11):
                future_high = highs[i+j]
                future_low = lows[i+j]
                if future_low <= stop_price:
                    losses += 1; gross_loss += (entry_price - stop_price); break
                elif future_high >= target_price:
                    wins += 1; gross_profit += (target_price - entry_price); break

    win_rate = (wins / total_trades * 100) if total_trades > 0 else 0
    profit_factor = (gross_profit / gross_loss) if gross_loss > 0 else (99.9 if gross_profit > 0 else 0.0)
    return win_rate, total_trades, profit_factor

# --- 4. ANALYSIS ENGINE (DUAL MODE) ---
def analyze_stock(ticker_input):
    ticker = ticker_input.upper()
    df = get_chunked_data(ticker, years_back=5)

    # Auto-add .JK if needed (Smart Detection)
    if df.empty and len(ticker) == 4 and ticker.isalpha():
        ticker = f"{ticker}.JK"
        df = get_chunked_data(ticker, years_back=5)

    if df.empty: return None, None

    # Currency Check (Rp vs $)
    currency_symbol = "$"
    try:
        # FIX: Remove session here too
        info = yf.Ticker(ticker).info
        if info.get('currency') == 'IDR': currency_symbol = "Rp"
    except: pass

    try:
        # --- A. MATH INDICATORS ---
        df['Mean'] = df['Close'].rolling(window=20).mean()
        df['StdDev'] = df['Close'].rolling(window=20).std()
        df['Z_Score'] = (df['Close'] - df['Mean']) / df['StdDev']

        # --- B. BACKTEST ---
        real_win_rate, total_trades, real_pf = calculate_backtest_metrics(df)
        years_label = f"{(df.index[-1] - df.index[0]).days / 365:.1f}Y"

        # Current Data
        curr = df.iloc[-1]
        price = curr['Close']
        z_score = curr['Z_Score']
        std = curr['StdDev']
        mean = curr['Mean']

        # --- C. LEVELS ---
        stop_loss = price - std
        target1 = price + std
        fair_value = mean

        # R:R Ratio
        risk = price - stop_loss
        reward = target1 - price
        rr_ratio = (reward / risk) if risk > 0 else 1.0

        # --- D. LOGIC: SWING VS INVESTOR ---

        # 🐇 Swing Strategy (Requires High Win Rate)
        if z_score < -2.0 and real_win_rate >= 50:
            swing_action = "✅ ENTER (Good Odds)"
        elif z_score < -2.0 and real_win_rate < 50:
            swing_action = "⚠️ RISKY (Low Win Rate)"
        elif z_score > 2.0:
            swing_action = "🔴 SHORT / SELL"
        else:
            swing_action = "💤 WAIT (No Setup)"

        # 🐢 Investor Strategy (Requires Deep Value)
        discount_pct = ((fair_value - price) / fair_value) * 100
        if z_score < -2.0:
            invest_action = "💎 STRONG BUY (Deep Value)"
            invest_advice = "Heavy Accumulation Zone"
        elif z_score < -1.0:
            invest_action = "✅ BUY (Undervalued)"
            invest_advice = "Normal Dollar Cost Average (DCA)"
        elif z_score > 2.0:
            invest_action = "🛑 PAUSE BUYING"
            invest_advice = "Too Expensive - Wait for Dip"
        else:
            invest_action = "⚪ HOLD / DRIP"
            invest_advice = "Fair Value - Wait for -1σ"

        # Formatting Helper
        def fmt(val):
            return f"{int(val):,}" if currency_symbol == "Rp" else f"{val:,.2f}"

        meme_key = "BUY" if z_score < -2 else ("SELL" if z_score > 2 else "WAIT")

        # --- E. THE REPORT ---
        report = f"""
📌 **ASSET:** {ticker} (Dual Analysis)
💰 **PRICE:** {currency_symbol} {fmt(price)}
📊 **Z-SCORE:** {z_score:.2f} σ (Vol: {(std/price)*100:.1f}%)
⏱️ **DATA:** {years_label} History | {pd.Timestamp.now().strftime('%Y-%m-%d')}
──────────────────────
🐇 **SWING TRADER (Short Term)**
*Goal: Quick Profit | Horizon: 1-2 Weeks*

🎯 **Signal:** {swing_action}
🎲 **Win Probability:** {real_win_rate:.1f}% (Based on {total_trades} trades)
⚖️ **R:R Ratio:** 1:{rr_ratio:.2f}
🛑 **Stop Loss:** {currency_symbol} {fmt(stop_loss)} (Strict)
🚀 **Target:** {currency_symbol} {fmt(target1)} (Mean Reversion)
──────────────────────
🐢 **INVESTOR (Long Term)**
*Goal: Accumulation | Horizon: Years*

💎 **Signal:** {invest_action}
🏷️ **Fair Value:** {currency_symbol} {fmt(fair_value)}
📉 **Discount:** You save {discount_pct:.1f}% vs Average
💡 **Strategy:** {invest_advice}
⚓ **Accumulation Zone:** Below {currency_symbol} {fmt(mean - std)}
──────────────────────
Bot Opinion: "{'Invest now, Trade later.' if real_win_rate < 50 and z_score < -2 else 'Good for both!'}"
"""
        return report, MEME_URLS[meme_key]

    except Exception as e:
        print(f"Error: {e}")
        return None, None

# --- 5. HANDLERS ---
async def analyze(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not context.args: return await update.message.reply_text("Usage: /analyze AAPL")
    ticker = context.args[0]
    await update.message.reply_text(f"🔍 Analyzing {ticker} for Trader vs Investor...")

    report, meme = analyze_stock(ticker)
    if report:
        await update.message.reply_photo(meme, caption=report, parse_mode=ParseMode.MARKDOWN)
    else:
        await update.message.reply_text("❌ Data Error. Try again.")

async def start(update, context):
    await update.message.reply_text("🤖 Dual-Mode Bot Ready! /analyze AAPL")

async def main():
    app = ApplicationBuilder().token(BOT_TOKEN).build()
    app.add_handler(CommandHandler("start", start))
    app.add_handler(CommandHandler("analyze", analyze))
    print("Bot is polling...")
    await app.run_polling()

if __name__ == '__main__':
    try: asyncio.run(main())
    except KeyboardInterrupt: pass

Bot is polling...
🔄 Deep Scan: AAPL (5 years)...
